In [1]:
import os
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np 
from scipy.stats import entropy
from scipy.stats import mannwhitneyu as mwu
import plotly.graph_objects as go

In [34]:
data_dir = '../Data/Clinical/'
data_dir2 = '../Result/'

# max_css = pd.read_csv( data_dir + "max_css_table.csv")
cl1 = pd.read_csv( data_dir + "clinical_data_1.csv", sep=',')
cl2 = pd.read_csv( data_dir + "clinical_data_2.csv", sep=',')
max_css = pd.read_csv(data_dir2 + "max_css_table.csv")

BCRHC01 = pd.read_csv(data_dir2 + 'aa_shannon_BCR_01_HC.csv')
BCRLC01 = pd.read_csv(data_dir2 + 'aa_shannon_BCR_01_LC.csv')
TCR01 = pd.read_csv(data_dir2 + 'aa_shannon_TCR_01.csv')
BCRHC02 = pd.read_csv(data_dir2 + 'aa_shannon_BCR_02_HC.csv')
BCRLC02 = pd.read_csv(data_dir2 + 'aa_shannon_BCR_02_LC.csv')
TCR02 = pd.read_csv(data_dir2 + 'aa_shannon_TCR_02.csv')

all = [BCRHC01, BCRLC01, TCR01, BCRHC02, BCRLC02, TCR02]
only01 = [BCRHC01, BCRLC01, TCR01]
only02 = [BCRHC02, BCRLC02, TCR02] 

BCRHC01.loc[BCRHC01['Time_point'] == 0, 'Time_point'] = 1

print(max_css.head())
print(BCRHC01.head())

            ID  severity
0  COV-CCO-001         2
1  COV-CCO-002         2
2  COV-CCO-003         2
3  COV-CCO-004         2
4  COV-CCO-006         1
             Sample  Row_number  Total_readcount  Shannon_diversity  \
0  COV-CCO-0411_IGG        2498            19342           6.727983   
1  COV-CCO-0411_IGM        7006            11251           8.179853   
2  COV-CCO-0412_IGG         348             1523           5.104862   
3  COV-CCO-0412_IGM        1183             1850           6.769221   
4  COV-CCO-0413_IGG        2743            12102           6.672048   

       PlnP2  Squared_Shannon  Time_point Type  
0  47.012921        45.265755           1  IGG  
1  69.480620        66.909994           1  IGM  
2  27.682505        26.059611           2  IGG  
3  46.744685        45.822355           2  IGM  
4  47.145421        44.516224           3  IGG  


C:\Users\정우빈\AppData\Local\Temp\ipykernel_15120\474078937.py:6: DtypeWarning: Columns (17,19,21,23,38,69,84,139,154,165,283,289,290,292,294,316) have mixed types. Specify dtype option on import or set low_memory=False.
  cl2 = pd.read_csv( data_dir + "clinical_data_2.csv", sep=',')


In [35]:
sev_dict = max_css['severity'].value_counts().to_dict()
print(sev_dict)

{1: 204, 2: 165, 3: 52, 4: 36, 5: 2}


In [53]:
for i in all:
    i['Severity'] = i['Sample'].str[0:11].map(max_css.set_index('ID')['severity'])
    i['Severity'] = i['Severity'].fillna(0)

print(TCR02['Severity'].value_counts())

Severity
4.0    94
3.0    66
0.0    44
2.0    16
Name: count, dtype: int64


In [55]:
severity_count = pd.DataFrame(columns=[0, 1, 2, 3, 4], index=['BCRHC01', 'BCRLC01', 'TCR01', 'BCRHC02', 'BCRLC02', 'TCR02'])ㄴ

for i in range(len(all)):
    for j in range(5):
        severity_count.iloc[i,j] = len(all[i][all[i]['Severity'] == j])

print(severity_count)

           0    1    2    3    4
BCRHC01  122  390  320    4    4
BCRLC01  122  390  320    4    4
TCR01    122  390  320    4    4
BCRHC02   76    0   30  115  161
BCRLC02   88    0   32  128  183
TCR02     44    0   16   66   94


In [78]:
### 모든 데이터프레임에서 severity가 각각 0, 1, 2, 3, 4인 경우에 대해 'Sample'리스트 작성

sev0,sev1,sev2,sev3,sev4 = [],[],[],[],[]

for i in all:
    sev0 += i[i['Severity'] == 0]['Sample'].str[0:11].unique().tolist()
    sev1 += i[i['Severity'] == 1]['Sample'].str[0:11].unique().tolist()
    sev2 += i[i['Severity'] == 2]['Sample'].str[0:11].unique().tolist()
    sev3 += i[i['Severity'] == 3]['Sample'].str[0:11].unique().tolist()
    sev4 += i[i['Severity'] == 4]['Sample'].str[0:11].unique().tolist()

### 중복 제거

sev0, sev1, sev2, sev3, sev4 = [list(set(sev)) for sev in [sev0, sev1, sev2, sev3, sev4]]

print(len(sev0), len(sev1), len(sev2), len(sev3), len(sev4))

81 85 72 10 17
